In [ ]:
import glob
from pathlib import Path
import os
import igl
import logging
from tqdm.auto import tqdm
import numpy as np


from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone, IFSelect_ItemsByEntity
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_NurbsConvert
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Extend.TopologyUtils import TopologyExplorer, WireExplorer
from OCCUtils.Topology import Topo, dumpTopology
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve, BRepAdaptor_Surface
from OCC.Core.BRep import BRep_Tool
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Pnt2d
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_NurbsConvert
from OCC.Core.BRepTools import breptools_UVBounds
from OCC.Core.ShapeFix import ShapeFix_Shape as _ShapeFix_Shape


def load_bodies_from_step_file(pathname, logger=None):
    assert pathname.exists()
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(str(pathname))
    if status == IFSelect_RetDone:  # check status
        shapes = []
        nr = 1
        try:
            while True:
                ok = step_reader.TransferRoot(nr)
                if not ok:
                    break
                _nbs = step_reader.NbShapes()
                shapes.append(step_reader.Shape(nr))  # a compound
                #assert not shape_to_return.IsNull()
                nr += 1
        except:
            logger.error("Step transfer problem: %i"%nr)
            print("No Shape", nr)
    else:
        logger.error("Step reading problem.")
        raise AssertionError("Error: can't read file.")

    return shapes
    
def save_mesh(output_pathname, body, rescale=True, filter_low=True):
    mesh = BRepMesh_IncrementalMesh(body, 0.95, False, 0.04, True)
    mesh.SetShape(body)
    mesh.Perform()
    
    top_exp = TopologyExplorer(body)
    brep_tool = BRep_Tool()
    faces = top_exp.faces()
    first_vertex = 0
    tris = []
    verts = []
    #print("Meshing")
    for face in faces:
        face_orientation_wrt_surface_normal = face.Orientation()
        location = TopLoc_Location()
        mesh = brep_tool.Triangulation(face, location)
        if mesh != None:
            # Loop over the triangles
            num_tris = mesh.NbTriangles()
            for i in range(1, num_tris+1):
                index1, index2, index3 = mesh.Triangle(i).Get()
                if not face_orientation_wrt_surface_normal:
                    tris.append([
                        first_vertex + index1 - 1, 
                        first_vertex + index2 - 1, 
                        first_vertex + index3 - 1
                    ])
                else:
                    tris.append([
                    first_vertex + index3 - 1, 
                    first_vertex + index2 - 1, 
                    first_vertex + index1 - 1
                ])
            num_vertices = mesh.NbNodes()
            first_vertex += num_vertices
            for i in range(1, num_vertices+1):
                verts.append(list(mesh.Node(i).Coord()))
    #print(output_pathname)
    if len(verts) > 0:
        verts = np.array(verts)
        tris = np.array(tris)

        if rescale:
            mi, ma = np.min(verts, axis=0), np.max(verts, axis=0)
            mean = (ma - mi)/2 + mi
            verts -= mean#np.mean(verts, axis=0)
            sf = np.max(ma-mi)
            rf = 200#np.random.randint(50, 100)
            verts = verts / sf * rf
        
        if filter_low:
            mi, ma = np.min(verts, axis=0), np.max(verts, axis=0)
            diff = ma - mi
            longest = np.max(diff)
            shortest = np.min(diff)
            if longest/shortest <= 2.0 and verts.shape[0] >= 5000:
                igl.write_triangle_mesh(output_pathname, verts, tris)
                
            #print(mean, np.min(verts, axis=0), np.max(verts, axis=0))
        else:
            igl.write_triangle_mesh(output_pathname, verts, tris)
    
def process(rang=[0, 10000], data_path="./data/conv/", output_path="./results_fixed"):
    data_dir = Path(data_path)
    output_dir = Path(output_path)
    os.makedirs(output_dir, exist_ok=True)
    
    # Glob step files in data dir
    extensions = ["stp", "step"]
    step_files = []
    for ext in extensions:
        files = [ f for f in data_dir.glob(f"*/*.{ext}")]
        step_files.extend(files)
        
    step_files = step_files[rang[0]:rang[1]]
    
    # Process step files
    pbar = tqdm(range(len(step_files)))
    for i in pbar:
        if os.path.isfile("%s/%s_mesh.obj"%(output_path, step_files[i].stem)):
            pbar.set_description("Skipping %s"%step_files[i])
        else:
            pbar.set_description("Processing %s"%step_files[i])
            sf = step_files[i]

            bodies = load_bodies_from_step_file(sf)
            for b_idx, body in enumerate(bodies):
                save_mesh("%s/%s_%04i_mesh.obj"%(output_path, sf.stem, b_idx), body)   
            
process(rang=[0, 1000], data_path="conv/", output_path="results_abc_l")

<ipython-input-2-e04034f3cab7>:105: RuntimeWarning: divide by zero encountered in double_scalars
  if longest/shortest <= 2.0 and verts.shape[0] >= 5000:
